<h1 style='text-align: center;'>Segmenting and Clustering Neighborhoods in Toronto</h1>

In [103]:
#import module that used in this case
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [36]:
#crawling data from wikipedia page
url         = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response    = requests.get(url)

soup        = BeautifulSoup(response.text, 'lxml')

In [39]:
#find table that we wanted
table       = soup.find('table', attrs={'class':'wikitable sortable'})

In [142]:
#iterate over row and append it to our list, then create a dataframe
postal_code = []
borough     = []
neighborhood= []

for row in table.findAll('tr'):
    col = row.findAll('td')
    if len(col) == 3:
        postal_code.append(col[0].find(text=True).replace('\n',''))
        borough.append(col[1].find(text=True).replace('\n',''))
        neighborhood.append(col[2].find(text=True).replace('\n',''))

dataframe = {'postal_code': postal_code, 'borough': borough, 'neighborhood': neighborhood}
df  = pd.DataFrame(dataframe)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_row', None)

In [143]:
#replace not assigned as nan then drop it
df['borough'].replace('Not assigned',np.nan, inplace=True)
df.dropna(axis=0, inplace=True)

In [146]:
#check neighborhood column is there a null column
df['neighborhood'].value_counts()

Downsview                                                                                                                                 4
Don Mills                                                                                                                                 2
Del Ray, Mount Dennis, Keelsdale and Silverthorn                                                                                          1
Christie                                                                                                                                  1
Dorset Park, Wexford Heights, Scarborough Town Centre                                                                                     1
University of Toronto, Harbord                                                                                                            1
Guildwood, Morningside, West Hill                                                                                                         1
Humewood-Cedarvale  

In [141]:
#check our dataframe shape
df.shape

(103, 3)